In [26]:
import numpy as np

import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri, IntVector, Formula
pandas2ri.activate()

%load_ext autoreload
%autoreload 2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
base = importr('base')
stats = importr('stats')
ergm = importr('ergm')
network = importr('network', on_conflict="warn",
                  robject_translations={
                      'as.tibble.network': 'as_tibble_network',
                      'as_tibble.network': 'as_tibble_network'
                  }
         )

/Users/hangjianli/.pyenv/versions/3.7.4/envs/leetcode/lib/python3.7/site-packages/rpy2/robjects/vectors.py:980: UserWarning: R object inheriting from "POSIXct" but without attribute "tzone".
  warnings.warn('R object inheriting from "POSIXct" but without '
/Users/hangjianli/.pyenv/versions/3.7.4/envs/leetcode/lib/python3.7/site-packages/rpy2/robjects/packages_utils.py:127: UserWarning: Conflict when converting R symbols in the package "network" to Python symbols: 
-as_tibble_network -> as_tibble.network, as.tibble.network
  warn(msg)


### Load the simulated $A^t$

In [48]:
filename = '../data/sim101.rds'
readRDS = robjects.r['readRDS']
res = readRDS(filename)
# res = np.array(df).astype(int)

In [49]:
resdic = dict(res.items())

In [50]:
resdic['coefs_pos']

array([[-1.,  1.,  2., -3.],
       [-2.,  2.,  1., -1.]])

In [51]:
df = resdic['nw']
df = np.array(df).astype(int)

In [52]:
resdic.get('desc')

'form: edge+mutual. diss: edge+mutual'


In [53]:
df[0,:5,:5]

array([[0, 0, 1, 0, 0],
       [1, 0, 1, 0, 1],
       [0, 0, 0, 0, 0],
       [0, 0, 1, 0, 1],
       [1, 0, 0, 0, 0]])

In [54]:
df.shape

(40, 20, 20)

## Initialize the params

In [62]:
num_nodes = df[0].shape[0]
# n = 20
T = len(df)
# num_changepts = 5

p_pos = 2 # edge + mutual
p_neg = 2 # edge + mutual
p = p_pos + p_neg

np.random.seed(42)
theta = np.random.normal(0, 0.5, size= (T , p)) 
z = np.copy(theta) 
u = np.zeros((T, p)) 


form_terms = ['edges', 'mutual']
diss_terms = ['edges', 'mutual']

In [63]:
init = {
    'theta': np.copy(theta),
    'z': z,
    'u': u
}

In [64]:
from mple_learn import stergmGraph

In [65]:
sim1 = stergmGraph(
    X = df,
    form_terms=form_terms,
    diss_terms=diss_terms,
    lam=[1],
    max_steps=100,
    newton_max_steps=100,
    admm_alpha=1,
    converge_tol=1e-5,
#     gfl_maxit=300,
#     gfl_tol=1e-5,
    verbose=1
)

In [66]:
theta_res, z, u, c = sim1.mple(initial_values=init, solver='newton', tau_inc=2, m=10)

[INFO] ADMM step #0
[INFO] Updating theta...
[INFO] Newton converged in: 0.03379034996032715.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.06599807739257812.
[INFO] Updating u...
[INFO] max mu :  0.9870993344291896
[INFO] dual_resnorm: 1.442049
[INFO] primal_resnorm: 0.345773
[INFO] convergence: 1.442049
-------------------------------------------------
[INFO] ADMM step #1
[INFO] Updating theta...
[INFO] Newton converged in: 0.022162914276123047.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.11795496940612793.
[INFO] Updating u...
[INFO] max mu :  0.9867157742631398
[INFO] dual_resnorm: 0.307523
[INFO] primal_resnorm: 0.221946
[INFO] convergence: 0.307523
-------------------------------------------------
[INFO] ADMM step #2
[INFO] Updating theta...
[INFO] Newton converged in: 0.01879405975341797.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.16305208206176758.
[INFO] Updating u...
[INFO] max mu :  0.991664625363314
[INFO] dual_resnorm: 0

[INFO] Group fused Lasso converged in: 0.39587903022766113.
[INFO] Updating u...
[INFO] max mu :  0.9959041916245204
[INFO] dual_resnorm: 0.007461
[INFO] primal_resnorm: 0.003170
[INFO] convergence: 0.007461
-------------------------------------------------
[INFO] ADMM step #23
[INFO] Updating theta...
[INFO] Newton converged in: 0.01314401626586914.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.41414594650268555.
[INFO] Updating u...
[INFO] max mu :  0.9959070940411394
[INFO] dual_resnorm: 0.007094
[INFO] primal_resnorm: 0.002992
[INFO] convergence: 0.007094
-------------------------------------------------
[INFO] ADMM step #24
[INFO] Updating theta...
[INFO] Newton converged in: 0.014757871627807617.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.39789485931396484.
[INFO] Updating u...
[INFO] max mu :  0.9959094092272978
[INFO] dual_resnorm: 0.006754
[INFO] primal_resnorm: 0.002837
[INFO] convergence: 0.006754
-------------------------------------------

[INFO] Group fused Lasso converged in: 0.443310022354126.
[INFO] Updating u...
[INFO] max mu :  0.9959184706586034
[INFO] dual_resnorm: 0.002821
[INFO] primal_resnorm: 0.001497
[INFO] convergence: 0.002821
-------------------------------------------------
[INFO] ADMM step #45
[INFO] Updating theta...
[INFO] Newton converged in: 0.01732778549194336.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.4403259754180908.
[INFO] Updating u...
[INFO] max mu :  0.9959185360496023
[INFO] dual_resnorm: 0.002702
[INFO] primal_resnorm: 0.001457
[INFO] convergence: 0.002702
-------------------------------------------------
[INFO] ADMM step #46
[INFO] Updating theta...
[INFO] Newton converged in: 0.013087987899780273.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.43352603912353516.
[INFO] Updating u...
[INFO] max mu :  0.9959186274956262
[INFO] dual_resnorm: 0.002592
[INFO] primal_resnorm: 0.001422
[INFO] convergence: 0.002592
----------------------------------------------

[INFO] Group fused Lasso converged in: 0.42278599739074707.
[INFO] Updating u...
[INFO] max mu :  0.9959224996283879
[INFO] dual_resnorm: 0.001155
[INFO] primal_resnorm: 0.000252
[INFO] convergence: 0.001155
-------------------------------------------------
[INFO] ADMM step #67
[INFO] Updating theta...
[INFO] Newton converged in: 0.013309001922607422.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.44754719734191895.
[INFO] Updating u...
[INFO] max mu :  0.9959225169142986
[INFO] dual_resnorm: 0.001110
[INFO] primal_resnorm: 0.000238
[INFO] convergence: 0.001110
-------------------------------------------------
[INFO] ADMM step #68
[INFO] Updating theta...
[INFO] Newton converged in: 0.011564970016479492.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.4583461284637451.
[INFO] Updating u...
[INFO] max mu :  0.9959225344876987
[INFO] dual_resnorm: 0.001066
[INFO] primal_resnorm: 0.000224
[INFO] convergence: 0.001066
-------------------------------------------

[INFO] Group fused Lasso converged in: 0.4243957996368408.
[INFO] Updating u...
[INFO] max mu :  0.9959224715719596
[INFO] dual_resnorm: 0.000563
[INFO] primal_resnorm: 0.000191
[INFO] convergence: 0.000563
-------------------------------------------------
[INFO] ADMM step #89
[INFO] Updating theta...
[INFO] Newton converged in: 0.009588956832885742.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.4294610023498535.
[INFO] Updating u...
[INFO] max mu :  0.995922331151046
[INFO] dual_resnorm: 0.000562
[INFO] primal_resnorm: 0.000189
[INFO] convergence: 0.000562
-------------------------------------------------
[INFO] ADMM step #90
[INFO] Updating theta...
[INFO] Newton converged in: 0.009319782257080078.
[INFO] Updating z...
[INFO] Group fused Lasso converged in: 0.42905592918395996.
[INFO] Updating u...
[INFO] max mu :  0.9959224791207031
[INFO] dual_resnorm: 0.000533
[INFO] primal_resnorm: 0.000188
[INFO] convergence: 0.000533
---------------------------------------------

In [69]:
init['theta'].shape

(40, 4)

In [68]:
np.arange(1, 40)[np.linalg.norm(np.diff(init['theta'], axis=0), ord=2, axis=1) > 3]

array([ 1,  4, 14, 29])

In [72]:
np.round(init['theta'],2)

array([[-5.08, -2.35,  5.5 , -1.81],
       [-0.85, -1.91,  2.7 , -2.3 ],
       [-1.05, -1.64,  2.69, -2.2 ],
       [-0.97, -1.35,  2.14, -1.98],
       [ 2.76,  0.56,  1.82,  1.46],
       [ 2.98,  0.68,  1.3 ,  1.8 ],
       [ 3.27,  0.9 ,  1.06,  1.88],
       [ 3.3 ,  0.91,  0.99,  1.83],
       [ 3.29,  0.88,  0.92,  1.92],
       [ 2.96,  0.45,  1.18,  2.1 ],
       [ 2.48, -0.13,  0.78,  2.16],
       [ 2.86,  0.19,  1.  ,  2.1 ],
       [ 3.12,  0.33,  0.42,  2.8 ],
       [ 2.65, -0.24,  0.85,  2.52],
       [ 3.14,  0.27, -0.94, -0.46],
       [ 3.07,  0.2 , -0.89, -0.43],
       [ 2.9 ,  0.18, -0.9 , -0.92],
       [ 2.7 ,  0.16, -1.13, -0.44],
       [ 2.47,  1.45, -0.59, -2.1 ],
       [ 1.55,  1.41, -0.69, -2.52],
       [ 2.16,  0.92, -0.66, -0.2 ],
       [ 2.25,  0.58, -0.67, -1.4 ],
       [ 2.25,  0.59, -0.69, -1.41],
       [ 2.22,  0.67, -0.52, -1.19],
       [ 2.15,  0.69, -0.51, -1.31],
       [ 1.85,  1.22, -0.81, -1.57],
       [ 1.95,  1.2 , -0.62, -1.38],
 

In [70]:
resdic['coefs_pos']
resdic['coefs_neg']

array([[-1.,  1.,  2., -3.],
       [-2.,  2.,  1., -1.]])

array([[ 2.07944154,  1.09861229, -0.69314718,  0.69314718],
       [-2.        ,  2.        , -1.        , -1.        ]])

In [71]:
np.append(resdic['coefs_pos'], resdic['coefs_neg'], axis=0).T

array([[-1.        , -2.        ,  2.07944154, -2.        ],
       [ 1.        ,  2.        ,  1.09861229,  2.        ],
       [ 2.        ,  1.        , -0.69314718, -1.        ],
       [-3.        , -1.        ,  0.69314718, -1.        ]])

### Test Newton's method

In [9]:
hess_f = sim1.theta_update_hess_f(theta)
grad_f = sim1.theta_update_grad_f(theta, z, u)

### Test fused lasso

In [48]:
import matlab.engine
eng = matlab.engine.start_matlab()
eng.addpath(eng.genpath('../gfl/'), nargout=0)

### Compute BIC

In [12]:
df.shape

(40, 20, 20)

In [13]:
yt0 = df[1,:,:]
yt1 = df[2,:,:]

In [14]:
ypos = np.logical_or(yt0, yt1).astype(int)
yneg = np.logical_and(yt0, yt1).astype(int)
nwpos = network.network(ypos)

In [15]:
form_terms = ['edges', 'mutual']
fml_form = Formula('nwf ~ ' + ' + '.join(form_terms))

In [20]:
fml_form.environment['nwf'] = nwpos

In [ ]:
lr_form = ergm.ergmMPLE(self.fml_form, output='array')

In [ ]:
        ypos = np.logical_or(yt0, yt1).astype(int)
        yneg = np.logical_and(yt0, yt1).astype(int)
        nwpos = network.network(ypos)
        nwneg = network.network(yneg)
        self.fml_form.environment['nwf'] = nwpos
        self.fml_diss.environment['nwd'] = nwneg
        
        # compute change statistic with correction
        lr_form = ergm.ergmMPLE(self.fml_form, output='array')
        lr_form_delta = np.array(lr_form.rx('predictor')).squeeze(axis=0)
        lr_diss = ergm.ergmMPLE(self.fml_diss, output='array')
        lr_diss_delta = np.array(lr_diss.rx('predictor')).squeeze(axis=0)